In [132]:
# Import necessary libraries
import requests
import pandas as pd
import time

In [133]:
# --- Configuration ---
API_KEY = 'a4ffeb15dbea8c018bf61eeefbaa348c7b63e9c2'  # Replace with your Census API Key
STATE_FIPS = '12'  # Florida FIPS code
YEARS = [str(year) for year in range(2012, 2025)]
POVERTY_VAR = "S1701_C03_001E"  # Poverty rate variable for ACS 1-year estimates


In [134]:
# Define the counties and their FIPS codes
county_fips = {
    'Alachua': '001', 'Baker': '003', 'Bay': '005', 'Bradford': '007', 'Brevard': '009',
    'Broward': '011', 'Calhoun': '013', 'Charlotte': '015', 'Citrus': '017', 'Clay': '019',
    'Collier': '021', 'Columbia': '023', 'De Soto': '027', 'Dixie': '029', 'Duval': '031',
    'Escambia': '033', 'Flagler': '035', 'Franklin': '037', 'Gadsden': '039', 'Gilchrist': '041',
    'Glades': '043', 'Gulf': '045', 'Hamilton': '047', 'Hardee': '049', 'Hendry': '051',
    'Hernando': '053', 'Highlands': '055', 'Hillsborough': '057', 'Holmes': '059', 'Indian River': '061',
    'Jackson': '063', 'Jefferson': '065', 'Lafayette': '067', 'Lake': '069', 'Lee': '071',
    'Leon': '073', 'Levy': '075', 'Liberty': '077', 'Madison': '079', 'Manatee': '081',
    'Marion': '083', 'Martin': '085', 'Miami-Dade': '086', 'Monroe': '087', 'Nassau': '089',
    'Okaloosa': '091', 'Okeechobee': '093', 'Orange': '095', 'Osceola': '097', 'Palm Beach': '099',
    'Pasco': '101', 'Pinellas': '103', 'Polk': '105', 'Putnam': '107', 'Santa Rosa': '113',
    'Sarasota': '115', 'Seminole': '117', 'St. Johns': '109', 'St. Lucie': '111', 'Sumter': '119',
    'Suwannee': '121', 'Taylor': '123', 'Union': '125', 'Volusia': '127', 'Wakulla': '129',
    'Walton': '131', 'Washington': '133'
}

In [135]:
# URL Template (corrected)
URL_TEMPLATE = "https://api.census.gov/data/{year}/acs/acs1/subject?get=NAME,{poverty_var}&for=county:*&in=state:{state}&key={key}"


In [136]:
# Function to fetch poverty data (fixed format)
def get_poverty_data(year, state_fips, poverty_var, api_key):
    url = URL_TEMPLATE.format(year=year, poverty_var=poverty_var, state=state_fips, key=api_key)
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"❌ Failed to fetch data for {year}. Status code: {response.status_code}")
        return None


In [137]:
# --- Collect Data ---
all_data = []

In [138]:
for year in YEARS:
    print(f"📅 Fetching data for {year}...")
    data = get_poverty_data(year, STATE_FIPS, POVERTY_VAR, API_KEY)
    if data:
        for row in data[1:]:  # Skip header
            county_name = row[0].replace(" County, Florida", "").strip()
            rate = row[1]
            if rate.replace('.', '', 1).isdigit():
                rate = f"{rate}%"
            else:
                rate = "0%"
            all_data.append([year, county_name, rate])
    time.sleep(1)

📅 Fetching data for 2012...
📅 Fetching data for 2013...
📅 Fetching data for 2014...
📅 Fetching data for 2015...
📅 Fetching data for 2016...
📅 Fetching data for 2017...
📅 Fetching data for 2018...
📅 Fetching data for 2019...
📅 Fetching data for 2020...
❌ Failed to fetch data for 2020. Status code: 404
📅 Fetching data for 2021...
📅 Fetching data for 2022...
📅 Fetching data for 2023...
📅 Fetching data for 2024...
❌ Failed to fetch data for 2024. Status code: 404


In [139]:
# Create DataFrame
df = pd.DataFrame(all_data, columns=["Year", "County", "Poverty_Rate"])

# Remove duplicates (keep first if any duplicates exist)
df = df.drop_duplicates(subset=["Year", "County"])

# Pivot to Year x County format
pivot_df = df.pivot(index="Year", columns="County", values="Poverty_Rate")

# Add full row for 2020 with "0%"
pivot_df.loc["2020"] = "0%"

# Fill any other missing values with "0%"
pivot_df = pivot_df.fillna("0%")

# Sort by year and county name
pivot_df = pivot_df.sort_index()
pivot_df = pivot_df[sorted(pivot_df.columns)]

# Save to Excel
output_file = "florida_poverty_data_2012_2024.xlsx"
pivot_df.to_excel(output_file, sheet_name="County-Level Poverty Data")

print(f"✅ Final data saved to '{output_file}'")

✅ Final data saved to 'florida_poverty_data_2012_2024.xlsx'


In [140]:
missing_counts = pivot_df.isna().sum()
print(missing_counts.sort_values(ascending=False))


County
Alachua         0
Martin          0
Monroe          0
Nassau          0
Okaloosa        0
Orange          0
Osceola         0
Palm Beach      0
Pasco           0
Pinellas        0
Polk            0
Putnam          0
Santa Rosa      0
Sarasota        0
Seminole        0
St. Johns       0
St. Lucie       0
Sumter          0
Volusia         0
Miami-Dade      0
Marion          0
Bay             0
Manatee         0
Brevard         0
Broward         0
Charlotte       0
Citrus          0
Clay            0
Collier         0
Columbia        0
Duval           0
Escambia        0
Flagler         0
Hernando        0
Highlands       0
Hillsborough    0
Indian River    0
Lake            0
Lee             0
Leon            0
Walton          0
dtype: int64


In [98]:
pivot_df = pivot_df.applymap(lambda x: f"{x:.1f}%" if pd.notnull(x) else "N/A")


In [102]:
# Save the DataFrame to an Excel file
output_file = "florida_poverty_data_2012_2024.xlsx"
try:
    if not pivot_df.empty:
        pivot_df.to_excel(output_file, sheet_name="County_City-Level Poverty Data", index=True)
        print(f"Data saved successfully to {output_file}")
    else:
        print("No data to save.")
except Exception as e:
    print(f"Error saving data to Excel: {e}")

Data saved successfully to florida_poverty_data_2012_2024.xlsx


In [100]:
print(pivot_df.shape)
print(pivot_df.head())


(12, 7)
County Alachua County Bay County Duval County Escambia County Franklin County  \
Year                                                                            
2012              N/A        N/A          N/A             N/A           22.2%   
2013              N/A        N/A          N/A           18.1%             N/A   
2014              N/A        N/A          N/A             N/A             N/A   
2015              N/A        N/A          N/A           15.8%             N/A   
2016              N/A        N/A        16.6%             N/A             N/A   

County Palm Beach County Washington County  
Year                                        
2012                 N/A               N/A  
2013                 N/A               N/A  
2014                 N/A             19.5%  
2015                 N/A               N/A  
2016                 N/A               N/A  


County
Bay County           11
Franklin County      11
Palm Beach County    11
Duval County         10
Escambia County      10
Washington County    10
Alachua County        9
dtype: int64
